# Support Vector Machine :  Problem 2
## Classification with Soft Margins

## Disclaimer
These exercises are based on the Stanford Machine Learning Course [CS229](http://cs229.stanford.edu) of Andrew Ng. The environment of the exercise have been tuned to the theory content taught at Ravensburg Weingarten University.

We are using the Python programming language. If you don't know Python or if you would like to refresh your memory, take a look at the [Python tutorial](http://docs.python.org/tut/).
We will mostly work with NumPy, the fundamental package for scientific computing in Python. Please read the [NumPy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html). In addition, the documention of MatPlotLib and Scipy lib can be found here: .[MatplotLib](https://matplotlib.org/). [Scipy](https://docs.scipy.org/doc/scipy/reference/tutorial/).

### 2.A Dataset 

In this exercise, you will be using support vector machines
(SVMs) with example 2D dataset. Experimenting with these datasets
will help you gain an intuition of how SVMs work and how to use a Gaussian
kernel with SVMs.


In [ ]:
# Used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
import matplotlib.pyplot as plt

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# for plotting grayscale
from PIL import Image

import pandas as pd

In [ ]:
df = pd.read_csv('''Read Data''')
X = df[['x','y']].values 
y = df['label'].values
(n,d) = X.shape

m = '''Size of dataset'''
print('There are total of {} points in dataset'.format(m))

In [ ]:
def plot(X,y):

    '''Code here'''

In [ ]:
plot(X,y)

## Soft margin SVM Primal Optimization


**Why soft margins are needed ?**

Normally, in case of outliers, the model tries to Overfit and generates a so-called 'hard margins'. In this case, we can use slack variable ($\xi_i$). The points are then allowed to have margin of slightly less than 1, but then the cost of objective function will increase by a factor of $C\xi_i$.

In this part of the exercise, you will try using different values of the $C$
parameter with SVMs. Informally, the $C$ parameter is a positive value that
controls the penalty for misclassified training examples. A large $C$ parameter 
tells the SVM to try to classify all the examples correctly. 

Given a training set $\{(x_i, y_i) \}$ with $1 \leq i \leq n, x_i \in \mathbb{R}^d, y_i \in \{+1, −1\}$, recall that the primal
SVM optimization problem is usually written as:

$$
\begin{align}
\min_{w,b}\quad & \lVert w \rVert^2 + C \sum_{i=1}^n \xi_i \\
\text{s.t.}\quad & y_i(w^Tx_i + b) \geq 1 - \xi_i\\
& \xi_i \geq 0
\end{align}
$$

In this part of the exercise, we will be using SVMs to do non-linear classification.
In particular, you will be using SVMs with Gaussian kernels on
datasets that are not linearly separable.

<img src='./graphic/1.png' width='650' height='650'>

### 2.B Gaussian Kernel
To find non-linear decision boundaries with the SVM, we need to first implement
a Gaussian kernel. You can think of the Gaussian kernel as a similarity function that measures the "distance" between a pair of examples, $(x, z)$ with $x,z\in\mathbb{R}^d$. The Gaussian kernel is also parameterized by a bandwidth parameter, $\sigma$, which determines how fast the similarity metric decreases (to $0$) as the examples are further apart. 

The Gaussian kernel function is defined as

$$
k(x,z) = \exp \Big( -\frac{\lVert x-z\rVert^2}{2\sigma^2} \Big)
$$

You should now complete the code in `gaussianKernel` to compute
the Gaussian kernel between two examples, ($x^{(i)}$, $x^{(j)}$) simply called `x` and `z` in the function.

In [ ]:
def gaussianKernel(x, z, sigma):
    '''Code here'''

**Test Kernel function**

In [ ]:
x = np.array([1, 2,  1])
z = np.array([0, 4, -1])

sigma = 2
sim = gaussianKernel(x, z, sigma)

print('Value of Gaussian Kernel is {}'.format(sim))
#this value should be about 0.325

Now implement a vectorized version of `gaussianKernel` such that it returns the Gram matrix  (sometimes also called a "kernel matrix"), $K\in \mathbb{R}^{n\times m}$ with

$$
K_{ij} = k(x^{(i)},z^{(j)})
$$

where $x^{(i)}$ is the $i$th row in $X$ and $z^{(j)}$ is the $j$th row in $Z$.

In [ ]:
def gaussianKernelVec(X, Z, sigma):
    
    '''Code here'''
    
    return K

**Test Vector Kernel Funct**

In [ ]:
XX = np.random.rand(3,2)
ZZ = np.random.rand(2,2)

K1 = gaussianKernelVec(XX, ZZ, sigma=.5)

print('Gram Matrix: {}'.format(K1))

### SVM Dual Form

Recall the primal support vector machine optimization problem above.
We can form the Lagrangian:
$$
\mathcal{L}(w,b,\xi,\alpha,\beta) = \frac{1}{2}w^Tw + C\sum_{i=1}^n \xi_i - \sum_{i=1}^n\alpha_i[y_i(x_i w+b) -w +\xi_i] - \sum_{i=1}^n\beta_i\xi_i
$$

Here, the $\alpha_i$'s and $\beta_i$'s are our Lagrange multipliers (constrained to be $\geq 0$).
We won’t go through the derivation of the dual again in detail, but after
setting the derivatives with respect to w and b to zero as before, substituting
them back in, and simplifying, we obtain the following dual form of the
problem:

$$
\begin{align}
\max_{\alpha} \quad & \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i,j=1}^n y_iy_j\alpha_i\alpha_j \langle x_i, x_j \rangle \\
\text{s.t.}\quad & 0\leq \alpha_i\leq C\\
& \sum_{i=1}^n \alpha_i y_i = 0
\end{align}
$$

We know that $w$ can be expressed in terms of the $\alpha_i$'s given by
$$
w = \sum_{i=1}^n \alpha_i y_i x_i,
$$
so that after solving the dual problem, we can use
$$
f(x) = w^Tx+b = \sum_{i=1}^n \alpha_i y_i \langle x_i, x \rangle +b
$$
to make our predictions.

Since the algorithm can be written entirely in terms of the inner products
$\langle x, z \rangle$, this means that we would replace all those inner products with
$\langle \phi(x), \phi(z) \rangle$. Specificically, given a feature mapping $\phi$, we define the corresponding
Kernel to be

$$
k(x, z) = \langle \phi(x), \phi(z) \rangle.
$$

Then, everywhere we previously had $\langle x, z \rangle$ in our algorithm, we could simply
replace it with $k(x, z)$, and our algorithm would now be learning using the
features $\phi$.

### 2.C Solving the SVM Dual Optimization problem with Quadratic Programming

The SVM dual problem can be optimized by quadratic programming. You need to download and install a third party library called "cvxopt". It is a python library for convex optimization problems.

**You may want to look into this : https://cvxopt.org/examples/tutorial/qp.html**

Quadratic programming is a general optimization routine to find a minimum for a problem specified by
$$
\begin{align}
\min_x\quad& \frac{1}{2} x^TPx + q^Tx\\
\text{s.t.}\quad & Gx \leq h\\
& Ax = b
\end{align}
$$
and we have to bring the SVM dual into this form.

In [ ]:
import cvxopt

def quadprog(P,q,G,h,A,b):
    
    sol = cvxopt.solvers.qp(cvxopt.matrix(P), 
                      cvxopt.matrix(q), 
                      cvxopt.matrix(G), 
                      cvxopt.matrix(h), 
                      cvxopt.matrix(A), 
                      cvxopt.matrix(b))
    return np.ravel(sol['x'])

#### The Quadratic Program
We write the SVM optimization problem as a Quadratic Program.
Let $K$ be the Gram matrix with $K_{i,j} = k(x_i,x_j)$ then

$$
\begin{align}
P & = yy^T \circ K\\
q & = [-1, -1, -1, \dotsc, -1] = [-1]\\
G & = [-I, I]^T\\
h & = [0, 1\cdot C]^T\\
A & = y\\
b & = 0
\end{align}
$$

where $[1]$ is a vector with $n$ ones and $I$ is the identity matrix of size $n\times n$.

The function `lagrange_multipliers` below calculates these quantities and calls `quadprog` to obtain the $\alpha$'s.

In [ ]:
def lagrange_multipliers(X, y, C, K):
    
    # Compute all matrices and then call cvxopt library
    
    '''Code here'''

    alpha = quadprog(P, q, G, h, A, b)
    return alpha

The optimal bias $b$ can be determined by KKT dual-complementarity conditions which quarantees that
$$
y_i (w^Tx_i + b) = 1 \quad\text{if}\quad 0 < \alpha_i < C
$$
and hence
$$
\begin{align}
y_i (w^Tx_i + b) &= 1\\
w^Tx_i + b &= y_i\\
b &= y_i - w^Tx_i\\
&= y_i - \sum_{j=1}^n \alpha_j y_j k(x_j, x_i)
\end{align}
$$
for any $0 < \alpha_i < C$.

Implement the function `bias` below which returns $b$.

In [ ]:
def bias(X, y, C, alpha, K):
    # for stability, we use not a single alpha but all alphas which fulfill the KKT critera.
    # And then average the result
    
    idx = (alpha > 1e-5) & (alpha < C)
    beta = '''Code here'''
    bs = '''Code here'''
    return np.mean(bs)

We now have function to calculate $\alpha$ and $b$. This allows us to implement the support vector machine learning routine. 
The function `svm_learn` below first computes $\alpha$ and $b$ and then returns a function 
$$
z\mapsto \sum_{i=1}^n \alpha_i y_i k(x_i, z) +b
$$
which is able to make predictions for new data point $z$.

In [ ]:
def svm_learn(X,y,C,kernel):
    
    # Kernel function is an argument here
    K = '''Code'''                         # kernel (Gram) matrix
    alpha = '''Code'''   # obtain lagrange multiplier 
    b = '''Code'''                 # calculate bias
    
    # Select support vectors.
    sidx = alpha > 1e-5
    SVs = X[sidx, :]    
    
    # SVM classification function
    def f(Z):
        K = kernel(SVs, Z) # compute kernel matrix
        beta = '''Code'''
        return '''Code'''

    return f

In [ ]:
def plot_decision(model, points):
    
    '''Code here'''

In [ ]:
sigma = 0.1
C = 1
svm = svm_learn(X, y, C, lambda X, Y: gaussianKernelVec(X, Y, sigma))

plot(X,y)
plot_decision(svm, X)

### 2.D Overfit ?

**Try with a high value of C and see that the model tries to classify the train data too nicely.**

In [ ]:
C = 100
svm = svm_learn(X, y, C, lambda X, Y: gaussianKernelVec(X, Y, sigma))

plot(X,y)
plot_decision(svm, X)

# What can you further contribute to this Notebook ?